Useful links for practice:
- [Python + Numpy tutorial from CS231n](https://cs231n.github.io/python-numpy)... . We use torch.tensor instead of numpy.array in this video. Their design (e.g. broadcasting, data types, etc.) is so similar that practicing one is basically practicing the other, just be careful with some of the APIs - how various functions are named, what arguments they take, etc. - these details can vary.
- [PyTorch tutorial on Tensor](https://pytorch.org/tutorials/beginne)...
- [Another PyTorch intro to Tensor](https://pytorch.org/tutorials/beginne)...

## Exercises:

### E01: 
Train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

#### Ans
First I'll attempt the solution using counting.

In [46]:
# load the data
words = open('/Users/amralaa/Desktop/nbs/zero2hero/names.txt').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

We want first to be able to extract trigrams from the data. We'll put them in a dict and keep counts of occurences.

In [47]:
t = {}
pairs = [] #get pairs of characters as we'll need later

for w in words:
    w = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(w,w[1:],w[2:]):
        #print(f'{ch1+ch2} {ch3}')
        t[(ch1+ch2, ch3)] = t.get((ch1+ch2, ch3), 0) + 1
        if (ch1+ch2) not in pairs:
            pairs.append(ch1+ch2)   

In [48]:
#t.items()

In [49]:
len(pairs)

601

In [50]:
# we can sort them by frequency same as we did in the lecture
#sorted(t.items(), key=lambda kv:-kv[1])

In [51]:
counts = sorted(t.items(), key=lambda kv:-kv[1])

The `dict.items()` method returns a view object. The view object is a list containing the key-value pairs of the dictionary, as tuples.

We then need to build a table (as torch tensor) which has 1st 2 ccs as rows and 3rd cc as columns such that each cell will give counts of how many times this single cc follows that pair of ccs.

In [124]:
# get unique characters
chars = ['.'] + sorted(list(set(''.join(words))))
chars[:10]

['.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

In [53]:
stoi_c = {}
for ix, ch in enumerate(chars):
    stoi_c[ch] = ix
#stoi_c

In [54]:
stoi_p = {}
for ix, pr in enumerate(pairs):
    stoi_p[pr] = ix
#stoi_p

In [55]:
import torch
import matplotlib.pyplot as plt

In [56]:
# now let's create the table / array / tensor
N = torch.zeros((len(pairs), len(chars)), dtype=torch.int32)
N.shape

torch.Size([601, 27])

In [57]:
# now, let's populate it with counts
for w in words:
    w = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(w, w[1:], w[2:]):
        ix1 = stoi_p[ch1+ch2]
        ix2 = stoi_c[ch3]
        N[ix1, ix2] += 1
        #print((ch1+ch2, ch3))

In [58]:
itos_c = {v:k for k,v in stoi_c.items()}
itos_p = {v:k for k,v in stoi_p.items()}

In [59]:
N.shape

torch.Size([601, 27])

Now, we have a table where each row represents a bigram and each column represents how many times a single letter follows that bigram in the dataset.

Remeber we want to make a model that given a bigram can predict the next letter that follows and can generate new names this way. In this model, we could for example start with ".e" and the model predict "m" then we have "em" and the model predicts "a" and so on. 

The question the model is asking each time then: given I see ".e", what is the probability I get "a" or "b" or "c" ..etc.
So, given a bigram, we want a probability distribution over the single characters. We use it to sample from them.
But, we have counts not probabilities. We thus need to generate probabilities from counts.

We have 601 x 27 tensor. We want a probabilty distribution over the single letters (columns) for each of the 601 bigrams (rows). So, we want to divide each cell in a row by the sum of that row.

In [225]:
# to get row sums
N.sum(1, keepdim=True).shape

torch.Size([601, 1])

In [226]:
P = N / N.sum(1, keepdim=True)

In [62]:
#confirm probability worked out
p.sum(1, keepdim=True)[:10]

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000]])

In [92]:
# a generator for the sampling
g = torch.Generator().manual_seed(42)

How to sample? 

We have rows of bigrams ".a, .b, .c, ... aa, ab, .. , zz"\
We have cols of singles "a, b, c .. ."\
The only possible start bigrams are ".a, .b, ...z"\
```
We need to initialize our draw by sampling from the possible 27 ".letter" bigrams out of the 601 bigram rows.
Suppose we pull ".e", we then get its row and sample the next single letter from it eg "m"
Our bigram then becomes "em" and we gets its row and sample a single letter from it.
We keep going until we hit "."
```

The Multinomial distribution is a generalization of the Binomial.\
Binomial counts the successes in a fixed number of trials that can only be categorized as success or failure.\
Multinomial keeps track of trials whose outcomes can fall into multiple categories, such as excellent, adequate, poor; or red, yellow, green, blue. (From:introduction-to-probability-by-joseph-k-blitzstein-and-jessica-hwang).

In [130]:
# frequencies of .letter bigrams (possible start bigrams)
alphabet = sorted(list(set(''.join(words))))
starts = {}
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        if ch1 == '.':
            starts[ch1+ch2] = starts.get(ch1+ch2, 0) + 1

In [190]:
#for tup in sorted(starts.items()):
    #print(tup[0], tup[1])

In [135]:
Nstarts = torch.tensor([tup[1] for tup in sorted(starts.items())])

In [142]:
pstarts = Nstarts / Nstarts.sum()

In [143]:
pstarts

tensor([0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273, 0.0184,
        0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029, 0.0512,
        0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [166]:
# let's create conversion dicts for these 
itos_bi = {i:'.'+ltr for i, ltr in enumerate(alphabet)}
#itos_bi

In [191]:
# now we can sample from starts and find out which bigram we sampled
ixr = torch.multinomial(pstarts, num_samples=1, replacement=True, generator=g).item()
bigram = itos_bi[ixr]
bigram

'.a'

In [202]:
# then we can get the row of that bigram
single = stoi_p[bigram] #next single
single

10

In [203]:
# confirm we have the right bigram
itos_p[10]

'.a'

In [227]:
# get its row
psingle = P[single]
psingle

tensor([0.0000, 0.0469, 0.0431, 0.0070, 0.0830, 0.0125, 0.0048, 0.0039, 0.0206,
        0.0349, 0.0061, 0.0170, 0.1433, 0.0871, 0.1413, 0.0023, 0.0039, 0.0020,
        0.1093, 0.0440, 0.0163, 0.0345, 0.0551, 0.0014, 0.0061, 0.0392, 0.0345])

In [228]:
# sample from it
ixc = torch.multinomial(psingle, num_samples=1, replacement=True, generator=g).item()
ixc

4

In [229]:
mono = itos_c[ixc]
mono

'd'

In [230]:
bigram[1] + mono

'ad'

In [231]:
# next bigram & repeat
stoi_p[bigram[1] + mono]

68

In [232]:
# in a loop
for _ in range(20):
    
    ixbi = torch.multinomial(pstarts, num_samples=1, replacement=True, generator=g).item() # ix of starting bigram
    bi = itos_bi[ixbi] # the bigram itself
    out = [bi] 

    while True:
        ixr = stoi_p[bi] # pull the row of the bigram
        p = P[ixr] # distribution over single letters following the bigram
        ixc = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item() # sample from it
        mono = itos_c[ixc] # get the single letter to add to output
        out.append(mono)
        bi = bi[1]+mono # new bigram (move 1 letter in prev bigram + add new single)
        if mono == '.':
            break
    print(''.join(out))

.chen.
.majacie.
.kweno.
.ne.
.kae.
.sy.
.mal.
.humarviv.
.ronitzperoelashalin.
.sus.
.amra.
.treantygellennirahlyaysha.
.lah.
.ka.
.mann.
.dakavaydeleigramickyi.
.sa.
.terea.
.myan.
.an.


'.chudi.'

### E02: 
Split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

### E03: 
Use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

### E04: 
We saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

### E05: 
Look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

### E06: 
Meta-exercise! Think of a fun/interesting exercise and complete it.